# A2A Client-Server

**IMPORTANT:** Start the A2A server first in a separate terminal:
```bash
uvicorn agentic_patterns.09_a2a.example_a2a_server:app --host 0.0.0.0 --port 8000
```

In [ ]:
import uuid
import asyncio
import rich
from fasta2a.client import A2AClient
from fasta2a.schema import Message, TextPart

## Connect to the A2A Server

In [ ]:
client = A2AClient(base_url="http://127.0.0.1:8000")
type(client)

## Discover Agent Capabilities

The Agent Card describes what the remote agent can do.

In [ ]:
import httpx

async with httpx.AsyncClient() as http:
    response = await http.get("http://127.0.0.1:8000/.well-known/agent-card.json")
    card = response.json()
    rich.print(card)

## Create and Send a Task

In [ ]:
prompt = "What is the sum of 40123456789 and 2123456789?"

message = Message(
    kind="message",
    role="user",
    parts=[TextPart(kind="text", text=prompt)],
    message_id=str(uuid.uuid4())
)

In [ ]:
response = await client.send_message(message=message)
rich.print(response)

task_id = response['result']['id']

## Poll for Task Completion

In [ ]:
from IPython.display import clear_output

while True:
    task = await client.get_task(task_id=task_id)
    state = task['result']['status']['state']
    
    clear_output(wait=True)
    rich.print(task)
    
    if state == "completed":
        break
    elif state == "failed":
        raise Exception("Task failed")
    
    await asyncio.sleep(0.2)

## Extract the Result

In [ ]:
artifacts = task['result'].get('artifacts', [])
for artifact in artifacts:
    for part in artifact.get('parts', []):
        if part.get('kind') == 'text':
            print(f"Result: {part['text']}")